# Spatial Temporal Forecasts - Assignment & Class notes


## Intro

This assignment and notes notes have been adapted from: 
Wikle, C. K., Zammit-Mangion, A., and Cressie, N. (2019), 
Spatio-Temporal Statistics with R, Boca Raton, FL: Chapman & Hall/CRC. 
The book and code (in `R`) are freely available online at 
[https://spacetimewithr.org](https://spacetimewithr.org)

In this module we consider a simple linear dynamical model
for Sea Surface Temperature (anomalies) data. The data are monthly anomalies
from a January 1970 - March 2003 climatology (averaged over time). 
and are gridded at a resolution of 2 x 2 degrees between 124E - 70W and 30S - 30N. 
These data were obtained from the NOAA Climate Prediction Center 
as available in the IRI/LDEO
Climate Data Library at Columbia University
[http://iridl.ldeo.columbia.edu/SOURCES/.CAC/](http://iridl.ldeo.columbia.edu/SOURCES/.CAC/). 

We have made this Sea Surface Temperature (SST) data available as three different 
plain-text files on Google drive. 
These are: `SSTlonlat.txt`, which contains the longitude and latitude
of each observation (2520 rows, 2 columns), `SSTlandmask.txt` which is a 2520 long
vector of 0/1 flags indicating whether the corresponding pixel is over land or 
water ("`1`" indicates land), and `SSTdata.txt` which contains 2520 rows and 399 columns. Each 
column corresponds to a 2520 vector of monthly SST measurements. 
For example, the observations for May 1992 for the 2520 locations 
can be found in column 269 ( `(1992 - 1970)*12 + 5 = 269`) (column 1 corresponds to January 1970).

Our goal is to use a simple dynamical model to obtain 6-month ahead forecasts. 
We will use data until April 1997 to predict the Oct 1997 anomalies. Since we have the Oct 1997 data, 
we will be able to evaluate the accuracy of the predictions. Note that Oct 1997 was an
El Nino month. 

To obtain 6-month forecasts we will consider two approaches: one 
is the one discussed in class, based on a 6-month lag model (so that we model the evolution of the
SST "surface" in 6-month steps). This document contains a detailed description of this
approach, along with the corresponding `MATLAB` code. 
The other approach is based on a 1-month lag model. 
As discussed in class, to estimate the parameters of either of these
models (lag-1 or lag-6) we will need to reduce the dimension of our data. 

The class notes below include all the code needed to reproduce the analysis 
discussed on Tuesday. The assignment (5 questions) can be
found at the end of this document.  You may want to consider adapting the code
in these class notes to work on the assignment. 

To submit your assignment please email a PDF version of your Notebook to Mohamed (mmmahmed@uvic.ca). 
Please rename your PDF as “firstname_lastname_week5".

## Class notes

### Load data

The following lines load the required data:

In [ ]:
SSTlonlat = importdata('SSTlonlat.txt',' ',1).data;
SSTlandmask = importdata('SSTlandmask.txt',' ',1).data;
SSTdata = importdata('SSTdata.txt',' ',1).data;

### Preprocessing

Preparing for our forecast exercise ahead, 
we now extract the Oct 1997 observations (this is our "target", we
will  later compare our predictions to them). The code below also builds our
training SST data set (with observations until April 1997),
ignoring land pixels. Note that April 1997
corresponds to column 328 in `SSTdata`, and Oct 1997
to column 334. At the same time we also remove the locations 
(rows in our data matrices above)
which correspond to land locations. 
This information is available in the "1-column" matrix `SSTlandmask`. 

In [ ]:
delete_rows = SSTlandmask == 1 ;
SST_Oct97 = SSTdata(~delete_rows, 334);
SSTdata = SSTdata(~delete_rows, 1:328);

### Compute EOF (principal components)

As discussed in class, to estimate the components of these simple
linear dynamic models, we need to reduce the dimension of our observations. 
The optimal basis on which to project our observations is given by the so-called 
Empirical Orthogonal Functions (EOF). 

Let $\tilde{\mathbf{Z}}_t = \mathbf{Z}_t - \hat{\mathbf{\mu}}_Z$ be 
the centered vectors
of $m = 2261$ water observations at time $t$, 
for $t=1, 2, \ldots, 328$, 
and let $\tilde{{\cal Z}}$ be the $328 \times 2261$
matrix that has $\tilde{\mathbf{Z}}_t$ in its $t$-th row. Then the lag-0
empirical spatial covariance matrix estimator is
$$
\hat{C}^{(0)}_\mathbf{Z} = 
\frac{1}{n_T-1} \tilde{{\cal Z}}^\top \tilde{{\cal Z}} = 
\frac{1}{n_T-1} \sum_{i=1}^{n_T} \tilde{\mathbf{Z}}_t \, 
\tilde{\mathbf{Z}}_t^\top
$$

Moreover, the eigenvalues of $\hat{C}^{(0)}_\mathbf{Z}$ 
can be computed from the Singular
Value Decomposition of the matrix $\tilde{{\cal Z}}$ (without having to compute
$\hat{C}^{(0)}_\mathbf{Z}$). Specifically: if
the SVD of the matrix $\tilde{{\cal Z}} / \sqrt{N_t -1}$ is 
$\tilde{{\cal Z}} = \mathbf{U} \, \mathbf{D} \,  \mathbf{V}^\top$, 
then the eigenvectors of 
$\hat{C}_\mathbf{Z}^{(0)} = \tilde{{\cal Z}}^\top \tilde{{\cal Z}} / (n_T-1)$ 
(which are to the EOFs)
are given by the columns of
$\mathbf{V}$. 

The next 4 blocks of code compute $\hat{C}^{(0)}_\mathbf{Z}$ as described above.

First we re-organize the data so that each row corresponds to the measurements
(across space) for one month, and columns correspond to locations.

In [ ]:
Z = SSTdata';

We now substract the vector of spatial means from each of the 
rows of `Z`, and divide this matrix by $\sqrt{n_T -1}$, where $n_T$ is the
number of observations available for each location
(in our case $n_T = 328$). First compute the vector of spatial means $\hat{\mathbf{\mu}}_\mathbf{Z} = (1/n_T)
\sum_{i=1}^{n_T} \mathbf{Z}_t$

In [ ]:
spat_mean = mean(Z, 1); 

and then substract $\hat{\mathbf{\mu}}_\mathbf{Z}$ from 
each row of `Z`:

In [ ]:
Zspat_detrend = Z - spat_mean; 

Finally, divide the centered matrix by the factor $\sqrt{n_T - 1}$:

In [ ]:
[nt m] = size(Z);
Zt = Zspat_detrend ./ sqrt(nt - 1);

Now, to compute the EOFs we use the Singular
Value Decomposition of ${\cal Z}$ as described above. In our code
this matrix is `Zt`. We can compute the SVD with the function
`svd()`. Specifically we need the 
$\mathbf{V}$ matrix which is returned in the third element
(in the code below, we're only interested in the result `Ev`):

In [ ]:
[U S Ev] = svd(Zt);

Since our dynamical model applies to the 
projections $\mathbf{\alpha}_t$ of the observations on the subspace generated
by the first $n=10$ EOFs, we compute these "new" $\mathbf{\alpha}_t$ vectors first. 
Recall that $\tilde{\mathbf{Z}}_t$ is the row vector of 
centered observations at time $t$, and hence its projection
coordinates (as a row vector $\alpha_t$)
are given by 
$$
\alpha_t = (Z_t - \mu_Z) \, \mathbf{V}_{n} = \tilde{Z}_t \, \mathbf{V}_{n}
$$
where $\mathbf{V}_n \in \mathbb{R}^{m \times n}$ contains the first $n$ 
columns of $\mathbf{V}$. We can compute the whole $n_T \times n$ matrix 
of $\alpha_t$ rows as
$$
\text{TS} = \tilde{{\cal Z}} \, \mathbf{V}_{n}
$$
For this analysis we will only use $n = 10$ EOFs (so we will reduce the dimension of 
the data from $m = 2261$ to $10$):

In [ ]:
n = 10;
TS = Zspat_detrend * Ev(:, 1:n);

Note that the $n$ coordinates of the projections ($\alpha_t$) of each of the 
monthly surfaces are in the rows of `TS` (which should have size 328 x 10). 

<!-- plot the mean -->
<!-- tmp <- cbind(SSTlonlat[-delete_rows,], spat_mean) -->
<!-- # 228 = (1989 - 1970)*12 -->
<!-- ggplot(tmp) + geom_tile(aes(x = lon, y = lat, fill = spat_mean)) + -->
<!--   scale_fill_distiller(palette = "Spectral", name = "degC") + -->
<!--   theme_bw() + coord_fixed() + -->
<!--   xlab("Longitude (deg)") + ylab("Latitude (deg)") -->

To check visually how much accuracy is lost when we replace our data 
$\mathbf{Z}_t$ with these
$n$-dimensional approximations, 
we use the observations of Jan 1989 
(which correspond to $t = 228$)
as an example. 
Its projection coordinates 
on the $n$-dimensional EOFs subspace are $\alpha_{228}$,
 the 228th row of `TS` (i.e. `TS(228,:)`). 
 Thus, the approximated vector of observations
is $\mathbf{V}_n \, \alpha_{228} + \mathbf{\mu}$. 
We can compute this approximated vector spatial measurements
with

In [ ]:
SSTreconstructed = Ev(:, 1:n) * TS(228, :)'  + spat_mean';

When you visualize the resulting `SSTreconstructed` and compare it with
the actual data `Z(:, 228)`, you will note that while the 
approximated SST surface reflects the
main trends of the observed
data, it also results in a much "smoother" (simpler) surface. We can expect
this "smoothing" artifact to affect the accuracy of the predictions to
be constructed from the lower-dimensional representations. 


### Estimate the dynamic model (6-month lag) 

A simple (lag-$\tau$) dynamic model (**if it were applied to the 
2261-dimensional data vectors** $\mathbf{Z}_t$) would be:
\begin{align*}
\mathbf{Z}_t &= \mathbf{\mu}_Z + \mathbf{Y}_t + \mathbf{\varepsilon}_t \\
\mathbf{Y}_t &= \mathbf{M} \, \mathbf{Y}_{t-\tau} + \mathbf{\gamma}_t
\end{align*}
where $\mathbf{\varepsilon}_t$ and $\mathbf{\gamma}_t$ are 
random errors with mean zero. 
The model above can also be written as
$$
\mathbf{Z}_t - \mathbf{\mu}_Z =  M ( \mathbf{Z}_{t-\tau} - \mathbf{\mu}_Z ) + \mathbf{\eta}_t
$$

**However**, recall that instead of using our high-dimensional vectors 
$\mathbf{Z}_t$, **we will work with the low dimensional projections** onto the
EOF space: $\mathbf{\alpha}_t$. Furthermore, note that 
$\sum_{t=1}^{n_T} \mathbf{\alpha}_t = \mathbf{0}$, so we 
can set $\mathbf{\mu}_{\mathbf{\alpha}} = \mathbf{0}$ and
get the linear dynamic model
$$
\mathbf{\alpha}_t = \mathbf{M} \, \mathbf{\alpha}_{t-\tau} + \mathbf{\eta}_t
$$
where $\mathbf{M} \in \mathbb{R}^{n \times n}$ is unknown, and 
the errors $\mathbf{\eta}_t$ are assumed to have mean zero and
unknown 
covariance matrix $C_\eta$. 

As discussed in class, a simple estimate the matrix of our
linear dynamical model is based on the observed lagged covariance
matrix. For a 6-month lag model we need to calculate the covariance
matrix of the coordinates of our observations 
when projected on the EOFs subspace ($\mathbf{\alpha}_t$) These were
stored in the rows of the `TS` object above. The lag-0 covariance matrix
can then be estimated with `TS' %*% TS / nT`.
Recall that 
$$
\hat{C}^{(0)}_\alpha = \frac{1}{n_T} \sum_{t=1}^{n_T} \alpha_t \alpha_t^\top
$$
(because $\sum_{t=1}^{n_T} \alpha_t = 0$, so we do not need to 
center them), and the $\tau$-lag empirical covariance matrix is:
$$
\hat{C}^{(\tau)}_\alpha = \frac{1}{n_T-\tau} \sum_{t=\tau+1}^{n_T} \alpha_{t} \alpha_{t-\tau}^\top
= \frac{1}{n_T-\tau} \sum_{t=1}^{n_T-\tau} \alpha_{t+\tau} \alpha_{t}^\top
$$
We now compute $\hat{C}^{(0)}_\alpha$ and $\hat{C}^{(\tau)}_\alpha$
with $\tau = 6$ (a 6-month lag model):

In [ ]:
tau = 6;
[nt d] = size(TS);
Cov0 = TS' * TS ./ nt; 
TStplustau = TS( (tau+1):nt, :); % TS with first tau time pts removed
TSt = TS(1:(nt-tau), :);    % TS with last tau time pts removed
Covtau = (TSt' * TStplustau)' ./ (nt - tau); 

As discussed in class, we can now use the relationship between $M$ and $C_\eta$ with 
$C^{(0)}_{\mathbf{\alpha}}$
and $C^{(\tau)}_{\mathbf{\alpha}}$
to estimate $\mathbf{M}$ and $C_\eta$. Recall that
$$
\hat{\mathbf{M}} = \hat{C}^{(\tau)}_{\mathbf{\alpha}} \, \left[ \hat{C}^{(0)}_{\mathbf{\alpha}} \right]^{-1}
\quad \text{and} \quad 
\hat{C}_\eta = \hat{C}^{(0)}_{\mathbf{\alpha}} - 
\hat{C}^{(\tau)}_{\mathbf{\alpha}} \, 
\left[ \hat{C}^{(0)}_{\mathbf{\alpha}} \right]^{-1} \,
\left. \hat{C}^{(\tau)}_{\mathbf{\alpha}} \right.^\top
$$
In `MATLAB`:

In [ ]:
C0inv = inv(Cov0); %<- solve(Cov0)
Mest = Covtau * C0inv;
Ceta = Cov0 - Covtau * C0inv * Covtau';

### Compute 6-months ahead predictions

Our simple dynamic model for the 
projections onto the
EOF space, $\mathbf{\alpha}_t$ is based on 6-month steps:
$$
\mathbf{\alpha}_t = \mathbf{M} \, \mathbf{\alpha}_{t-6} + \mathbf{\eta}_t
$$
Thus, we can predict the coordinates on the EOF space of 
the Oct 1997 values $\alpha_{334}$ by transformingt those 
of April 1997: $\alpha_{328}$. In symbols:
$$
\hat{\mathbf{\alpha}}_{334} = \hat{\mathbf{M}} \, \mathbf{\alpha}_{328}
$$
The code is simply:

In [ ]:
alpha_forecast = Mest * TS(328, :)';

The corresponding spatial predictions will be
$$
\mathbf{Z}^{(R)}_{334} = \mathbf{V}_{n} \, \hat{\mathbf{\alpha}}_{334}
+ \hat{\mathbf{\mu}}_Z
$$

In [ ]:
spatial_forecast = Ev(:, 1:n) * alpha_forecast + spat_mean';

Note that, assuming that $\hat{\mathbf{M}}$ is "known", we can estimate the
covariance matrix of the estimated $\hat{\mathbf{\alpha}}_{334}$ as
$$
\hat{C}_{\mathbf{\alpha}} = \hat{\mathbf{M}} \, \hat{C}_{\mathbf{\alpha}}^{(0)} \, 
\hat{\mathbf{M}}^\top + \hat{C}_{\mathbf{\eta}}
$$

In [ ]:
C = Mest * Cov0 * Mest' + Ceta;

and the estimated standard errors for the predictions in the vector 
$\hat{\mathbf{Z}}_{334}$ will be the square root of the diagonal 
elements of the matrix: 
$$
\mathbf{V}_n \, \hat{C}_{\mathbf{\alpha}} \, \mathbf{V}_n^\top
$$
We compute rough pixel-wise confidence bounds for
the predictions as twice their estimated standard errors:

In [ ]:
forecast_bounds = 2 .* sqrt(diag(Ev(:, 1:n) * C * Ev(:, 1:n)'));

<!-- To visualize the resulting prediction, we construct a -->
<!-- single object in `R` that contains the lat / lon -->
<!-- coordinates of each pixel, the actual Oct 1997 observations, -->
<!-- the Oct 1997 predictions, and their associated -->
<!-- approximate confidence bounds:  -->
We now visualize these. The actual Oct 1997 observations
and the predictions:

<!-- ```{r pred0.6, fig.show="hold", out.width="50%", echo=FALSE} -->
<!-- tmp <- SSTlonlat -->
<!-- tmp$pred <- NA -->
<!-- water <- which(SSTlandmask == 0) -->
<!-- tmp$obs[water] <- SST_Oct97 -->
<!-- tmp$pred[water] <- spatial_forecast  -->
<!-- tmp$forecast_bounds[water] <- forecast_bounds -->
<!-- ggplot(tmp) + -->
<!--   geom_tile(aes(lon,lat,fill=obs)) + -->
<!--   scale_fill_distiller(palette = "Spectral", name = "degC") + -->
<!--   theme_bw() + coord_fixed() + ggtitle("Observed Oct 1997") -->
<!-- ggplot(tmp) + -->
<!--   geom_tile(aes(lon,lat,fill=pred)) + -->
<!--   scale_fill_distiller(palette = "Spectral", name = "degC") + -->
<!--   theme_bw() + coord_fixed() + ggtitle("Predicted Oct 1997") -->
<!-- ``` -->

And finally, the associated standard errors:


<!-- ```{r pred0.7, out.width="50%", echo=FALSE} -->
<!-- ggplot(tmp) + -->
<!--   geom_tile(aes(lon,lat,fill=forecast_bounds)) + -->
<!--   scale_fill_distiller(palette = "Spectral", name = "degC") + -->
<!--   theme_bw() + coord_fixed() + ggtitle("Approx. error bounds for Oct 1997 predictions") -->
<!-- ``` -->


## Assignment 

1. Consider a lag-1 linear dynamic model 
and estimate the corresponding
operator (matrix) $\mathbf{M}$ 
using the same $n = 10$ EOF's we 
used in class. 
Compute predictions from Oct 1997
using data until April 1997, and the
corresponding pixel-wise standard errors. 
2. Compare, visually, your new forecast 
with the one obtained in class. What do you 
observe? Which method (if any) appears to yield
better predictions? In which sense are they 
better? 
3. Consider a higher-dimensional approximation to the
data using $n = 100$ EOF's. This will result
in better
approximations (the $\mathbf{Z}_t^{(R)}$'s 
will be closer to the $\mathbf{Z}_t$'s). 
Confirm (visually) that this is the case 
using the Jan 1989 observations. 
5. Repeat 1 and 2 above but using $n = 100$ EOFs.
6. Compare, visually, your new forecast 
with the one obtained in 2 above. Discuss.